In [1]:
import random
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
from collections import Counter
warnings.filterwarnings(action='ignore')

In [2]:
off_buyam_normal_final = pd.read_csv('./off_buyam_normal_final.csv')
off_buycnt_normal_final = pd.read_csv('./off_buycnt_normal_final.csv')
purchase_offline = pd.read_csv('./purchase_offline_nofood.csv')

## 협업필터링

In [3]:
# (target_df = 여기서는 normal 그룹의 구매금액 합, cid= 원하는 고객의 id, n_person=뽑고 싶은 유사한 고객의 수)
def get_sim_custs(target_df, cid, n_person):
    cluster_num = target_df[target_df['cust_id']==cid]['cluster'].values[0] # 원하는 고객의 클러스터 번호 추출
    sim_cluster = target_df[target_df['cluster']==cluster_num].copy()       #  같은 클러스터 고객들 추출
    n_df = sim_cluster.iloc[:,:-1].set_index('cust_id')               
    norm_values = n_df.values
    corr = np.corrcoef(norm_values)
    corr_df = pd.DataFrame(corr, index=n_df.index, columns=n_df.index)    # 같은 클러스터 고객들 상관계수 df
    sorted_df = corr_df[cid].sort_values(ascending=False)
    top_n_df = sorted_df[sorted_df<1][:n_person]
    res = top_n_df[top_n_df.values>0.5]  #피어슨 계수가 0.5 이상인 고객만
    return res

## 연관규칙을 이용한 lift가 가장 높은 상품 추출

In [4]:
# 구매내역 추출 함수
# (target_df = 여기서는 normal 그룹의 구매금액 합 데이터 프레임, cid= 원하는 고객의 id)
def get_buy_list(target_df,cid):
    buy_df = target_df[target_df['cust_id']==cid].iloc[:,1:-8]
    buy_bool = buy_df.values > 0
    buy_list = buy_df.columns[buy_bool[0]].to_list()
    return buy_list

In [5]:
# 고객별 하루 단위로 구매한 전체 리스트 (하루에 2개 이상 구매 기준)

# target_df=원하는 집단의 구매내역 데이터 프레임, cid= 구매내역을 알고싶은 고객id,구매내역을 저장할 list 변수
def get_tot_list(target_df, cid, prch_list):
    cust = target_df[target_df['cust']==cid]
    temp = cust[['clac_hlv_nm','de_dt']].drop_duplicates()
    res = temp.groupby('de_dt').count()
    date_list = res[res['clac_hlv_nm'] > 1].index.to_list() # 하루에 2개 이상 구매했을 때만
    
    for date in date_list:
        ctg_list = temp[temp['de_dt']==date]['clac_hlv_nm'].to_list()
        prch_list.append(ctg_list)

In [6]:
# 리프트 구하기
# tot_list = 전체 구매내역 리스트, x= 상품1, y=상품2
def get_lift(tot_list, x, y):
    try:
        tot = len(tot_list)
        x_cnt = 0
        y_cnt = 0
        xy_cnt = 0

        for val in tot_list:
            if (x in val) and (y in val):
                x_cnt +=1
                y_cnt +=1
                xy_cnt +=1
            else:
                if x in val:
                    x_cnt +=1
                if y in val:   
                    y_cnt +=1

        x_support = x_cnt/tot
        y_support = y_cnt/tot
        xy_support = xy_cnt/tot
        lift = xy_support/(x_support*y_support)

        return lift
    except:
        return -100
    
    
# 특정상품 support 구하기
# tot_list = 전체 구매내역 리스트, x=타겟 상품
def get_support(tot_list, x):
    try:
        tot = len(tot_list)
        x_cnt = 0

        for val in tot_list:
            if x in val:
                x_cnt +=1
                
        x_support = x_cnt/tot
        return x_support
    except:
        return -100

In [7]:
def top_lift_prd(target_prd, tot_list, ctg_list):
    res_dict = {}
    crgr_copy = ctg_list.copy()
    crgr_copy.remove(target_prd)
    for ctg in crgr_copy:
        res_dict[ctg]=get_lift(tot_list,target_prd, ctg)
    
    if (res_dict[max(res_dict, key=res_dict.get)] > 1):
        return max(res_dict, key=res_dict.get)
    else:
        return None

## 최종 추천 카테고리 추출

In [8]:
def get_rec_prd(cid, n_person):
    rec_prd = []
    support_dict = {}
    target_people = get_sim_custs(off_buyam_normal_final, cid, n_person).index.to_list()
    target_people.append(cid)
    
    for person in target_people:
        buy_list = get_buy_list(off_buycnt_normal_final, person) # 해당 고객이 구매한 내역
        cluster_num = off_buycnt_normal_final[off_buycnt_normal_final['cust_id']==person]['cluster'].values[0] # 개수 기반 클러스터 번호 추출
        sim_cluster_ids = off_buycnt_normal_final[off_buycnt_normal_final['cluster']==cluster_num]['cust_id'].to_list() # 개수 기반 동일 클러스터 고객들 id
        prch_df = purchase_offline[purchase_offline['cust'].isin(sim_cluster_ids)] # 유사 클러스터집단 구매내역 df
        ctgr_set = set(prch_df['clac_hlv_nm'].to_list()) #카테고리 set
        prch_list = []  # 유사 클러스터 고객별 날짜별 묶음 내역 -> 연관규칙 계산에 활용
        
        # 연관규칙 계산에 활용할 날짜별 구매내역 생성
        for cid in sim_cluster_ids:
            get_tot_list(prch_df, cid, prch_list)
            
        # lift가 가장 높은 상품을 추천상품 내역에 append
        for prd in buy_list:
            final_prd = top_lift_prd(prd, prch_list,ctgr_set)
            if final_prd != None:
                rec_prd.append(final_prd)
                support_dict[final_prd] = get_support(prch_list,final_prd)
            
    cnt = Counter(rec_prd)
    count_list= []
    for i in range(len(cnt.most_common())):
        count_list.append(cnt.most_common()[i][1])
    top_cnt = count_list.count(cnt.most_common()[0][1])  # 빈도수가 가장 높은 카테고리 중복개수
    
    if top_cnt == 1:                                     # 중복이 없으면 가장 높은거 1개 추출
        return cnt.most_common()[0][0]  
    else:                                                # 중복이 있다면 support가 가장 높은 것을 추출
        top_list = []
        for i in range(top_cnt):
            top_list.append(cnt.most_common()[i][0])
        
        top_support = {}
        for prd in top_list:
            top_support[prd] = support_dict[prd]
        return max(top_support, key=top_support.get)

In [9]:
get_rec_prd('M542006779', 10)

'여행/레저서비스'